# Compare vaccination strategies. $e$=0.7

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib as mpl
import datetime
from experiments import get_experiments_results
from env_var import EPIDEMIC, MAPPINGS, EXPERIMENTS
from matplotlib.lines import Line2D

%load_ext autoreload
%autoreload 2

mpl.rcParams['text.usetex'] = True
mpl.rcParams['figure.dpi']= 175
plt.style.use('ggplot')

In [2]:
# Simulation parameters
num_age_groups = EXPERIMENTS['num_age_groups']
region = EXPERIMENTS['region']
T = EXPERIMENTS['simulate_T']
init_vacc = EXPERIMENTS['init_vacc']
taus = EXPERIMENTS['taus']
u = EXPERIMENTS['vaccines_per_day']
r_experiments = EXPERIMENTS['r_effs']
t0 = EXPERIMENTS['t0']
e = EPIDEMIC['e']
strategies = EXPERIMENTS['strategies']
u_offset = EXPERIMENTS['u_offset']

# Plot parameters
plot_interval = 10
region_labels = EPIDEMIC['region_order'][region]
age_labels = MAPPINGS['age_groups'][num_age_groups]['names']
color_palette = ["#999999", "#E69F00", "#56B4E9", "#009E73", "#F0E442", "#0072B2", "#D55E00", "#CC79A7"]
len_bar = 5
sep_bet_group = 4*len_bar
legend_width = 4
plot_metrics = ['mortality', 'new hospitalizations', 'incidence']
plot_baseline = 'Pop'
col_order = ['Hosp', 'Inc', 'Inc+Hosp', 'Pop+Hosp', 'Pop+Inc', 'Pop+Inc+Hosp', 'Optimized']

begin = datetime.datetime.strptime(t0, '%Y-%m-%d')
ts = [begin + datetime.timedelta(days=day) for day in range(T)]
print('Simulation ends: %s' % (ts[-1]))
t2 = int(T-1-u_offset)
t1 = t2-30

Simulation ends: 2021-08-10 00:00:00


In [3]:
exp_results = get_experiments_results(num_age_groups=num_age_groups,
                                      region=region,
                                      init_vacc=init_vacc,
                                      strategies=strategies,
                                      u=u,
                                      T=T,
                                      r_experiments=r_experiments,
                                      t0=t0,
                                      e=e,
                                      taus=taus)

Beginning experiments.
Parameters:
Regions: erva.
Number of age groups: 9.
Number of vaccines per day: 30000.
t0: 2021-04-18.
T: 115.
Vaccine efficacy (e): 0.7.
Rs to try: [0.75, 1.0, 1.25, 1.5].
Taus to try: [0.0, 0.5, 1.0].
Initialize with vaccinated people: True.
Strategies:
[([1, 0, 0], 'Pop'), ([0, 0, 0], 'No vaccination'), ([0.3333333333333333, 0.3333333333333333, 0.3333333333333333], 'Pop+Inc+Hosp'), ([0.5, 0, 0.5], 'Pop+Hosp'), ([0.5, 0.5, 0], 'Pop+Inc'), ([0, 0.5, 0.5], 'Inc+Hosp'), ([0, 0, 1], 'Hosp'), ([0, 1, 0], 'Inc'), (True, 'Optimized')].

Found file: /Users/aponcedeleonch/Documents/Aalto/Courses/MasterThesis/Code/jeta/out/erva_R_0.75_tau_0.0_t0_2021-04-18_T_115_u_op.npy
Found file: /Users/aponcedeleonch/Documents/Aalto/Courses/MasterThesis/Code/jeta/out/erva_R_1.0_tau_0.0_t0_2021-04-18_T_115_u_op.npy
Found file: /Users/aponcedeleonch/Documents/Aalto/Courses/MasterThesis/Code/jeta/out/erva_R_1.25_tau_0.0_t0_2021-04-18_T_115_u_op.npy
Found file: /Users/aponcedeleonch/Docu

In [4]:
def plot_results(ax, r, tau, metric, exp_results, T, region, region_labels, u_offset,
                 plot_interval, baseline=plot_baseline, skip_no_vacc=True, out_sort=True):
    # Check the index of the region we are plotting
    try:
        region_idx = region_labels.index(region)
    except ValueError:
        if region == 'Finland':
            region_idx = -1
        else:
            raise ValueError('Wrong region selected')
    
    # Get only the elements starting from the offset
    x = np.arange(T)
    x = x[u_offset:]

    # Look for the baseline result for the current r, tau
    results = exp_results[r][tau]
    baseline_all_time = None
    for label, label_level in results.items():
        if label == baseline:
            baseline_results = label_level['results'][metric]
            # Sum across all age groups
            baseline_all_time = baseline_results.sum(axis=0)
            if region_idx == -1:
                # Sum across all regions
                baseline_all_time = baseline_all_time.sum(axis=0)
            else:
                baseline_all_time = baseline_all_time[region_idx, :]
            baseline_all_time = baseline_all_time[u_offset:]
    
    assert baseline_all_time is not None
    assert baseline_all_time.shape[0] == len(x)
    
    # Iterate over all the strategies
    totals = []
    label_i = 0
    for label, label_level in results.items():
        if label == baseline:
            continue

        if label == 'No vaccination' and skip_no_vacc:
            continue
            
        metric_results = label_level['results'][metric]
        # Sum across all age groups
        metric_all_time = metric_results.sum(axis=0)
        if region_idx == -1:
            # Sum across all regions
            metric_all_time = metric_all_time.sum(axis=0)
        else:
            metric_all_time = metric_all_time[region_idx, :]
        
        metric_all_time = metric_all_time[u_offset:]
        
        assert metric_all_time.shape[0] == len(x)
        
        # Get the relative change wrt to the baseline
        plot_metric = metric_all_time/baseline_all_time
        plot_metric = np.nan_to_num(plot_metric, nan=1)
        plot_metric = plot_metric - 1
        plot_metric = plot_metric*100
        
        # Plot the temporal line
        color = label_i % len(color_palette)
        ax.plot(x, plot_metric, label=label, color=color_palette[color])
        
        # Get also the result over all time so then produce sorted results
        baseline_policy = baseline_results.sum()
        total_policy = metric_results.sum()
        rel_total = ((total_policy/baseline_policy)-1)*100
        total = (label, rel_total)
        totals.append(total)
        label_i += 1
    
    # Print the sorted results
    if out_sort:
        sorted_totals = sorted(totals, key=lambda x: x[-1])
        for total in sorted_totals:
            print('%s: %s' % (total[0], total[1]))
    
    # Plot the labels
    ax.tick_params(labelsize=16)
    ax.set_xlabel('Days after start time')
    ax.set_ylabel('Relative change in {0} (\%)'.format(metric))
    ax.set_title('Relative change of %s in %s compared to %s.\n R=%s. tau=%s' % (metric, region, baseline, r, tau))
    ax.legend()
    ax.xaxis.set_major_locator(ticker.MultipleLocator(plot_interval))
        
    return ax

# Deaths

In [5]:
r_show, tau_show = 1.5, 0.5

In [6]:
fig, ax = plt.subplots(figsize=(12, 10))
ax = plot_results(ax, r=r_show, tau=tau_show, metric='mortality', exp_results=exp_results, T=T,
                  region='Finland', region_labels=region_labels, plot_interval=plot_interval,
                  u_offset=u_offset)

# Hospitalizations

In [7]:
fig, ax = plt.subplots(figsize=(12, 10))
ax = plot_results(ax, r=r_show, tau=tau_show, metric='new hospitalizations', exp_results=exp_results, T=T,
                  u_offset=u_offset, region='Finland', region_labels=region_labels, plot_interval=plot_interval)

# Infections

In [8]:
fig, ax = plt.subplots(figsize=(12, 10))
ax = plot_results(ax, r=r_show, tau=tau_show, metric='incidence', exp_results=exp_results, T=T,
                  u_offset=u_offset, region='Finland', region_labels=region_labels, plot_interval=plot_interval)

# Saving figures and generating latex

In [9]:
def save_and_generate_timeplot(r_experiments, taus, region_labels, metrics, exp_results, skip_no_vacc, out_sort, dpis,
                            u_offset, T, img_dir, plot_interval, baseline, file_format, latex_template):
    os.makedirs(img_dir, exist_ok=True)
    plot_regions = ['Finland']+ region_labels
    # Iterate over all the results
    for r in r_experiments:
        for tau in taus:
            for metric in metrics:
                # Plot in a single figure the values for Finland and all the regions
                fig, axs = plt.subplots(2, 3,
                                        sharex=True,
                                        figsize=(16, 11))
                axs = axs.flat
                # Iterate across all regions
                for region, ax in zip(plot_regions, axs):
                    ax = plot_results(ax, r=r, tau=tau, metric=metric, exp_results=exp_results, T=T,
                                      region=region, region_labels=region_labels,
                                      skip_no_vacc=skip_no_vacc, u_offset=u_offset, plot_interval=plot_interval,
                                      baseline=baseline, out_sort=out_sort)
                    # Remove the legends and axis labels, they are going to be in the figure
                    handles, labels = ax.get_legend_handles_labels()
                    ax.get_legend().remove()
                    x_lab = ax.get_xlabel()
                    ax.set_xlabel('')
                    y_lab = ax.get_ylabel()
                    ax.set_ylabel('')
                    ax.set_title('%s' % (region, ), y=0.93, fontsize=18)
                # Include in the figure the axis labels and legend
                fig.text(0.5, -0.02, x_lab,
                         ha='center', fontsize=24)
                fig.text(-0.02, 0.5, y_lab,
                         va='center', rotation='vertical', fontsize=24)
                
                ncols = round(len(labels)/2)
                legend = fig.legend(handles, labels, fontsize=20,
                                    loc='upper center', bbox_to_anchor=(0.5, 1.1),
                                    framealpha=0.5, ncol=ncols)
                
                fig.tight_layout()
                
                name = 'r%s_tau%s_metric_%s' % (r, tau, metric)
                # Save the figure
                fig_name = name + '.' + file_format 
                fig_path = os.path.join(img_dir, fig_name)
                fig.savefig(fig_path, format=file_format, dpi=dpis, bbox_inches='tight')
                # Construct the latex code for this figure
                latext_text = latex_template.format(fig_name=fig_name,
                                                    metric=metric,
                                                    baseline=baseline,
                                                    name=name,
                                                    r=r,
                                                    tau=tau)
                print(latext_text)
                
                

In [10]:
latex_template = """
    \\begin{{figure}}[p]
      \\centering
      \\includegraphics[width=\\textwidth]{{img/{fig_name}}}
      \\caption{{Relative change in {metric} for all vaccination strategies with respect to vaccination strategy {baseline}.
      For this scenario, the effective reproduction number $\\Reff = {r}$ and the mobility value $\\tau = {tau}$.}}
      \\label{{fig:{name}}}
    \\end{{figure}}
"""
save_and_generate_timeplot(r_experiments=r_experiments, taus=[0.5], region_labels=region_labels, metrics=plot_metrics,
                        exp_results=exp_results, skip_no_vacc=True, u_offset=u_offset, file_format='pdf',
                        T=T, img_dir=os.path.join('out', 'img'), latex_template=latex_template,
                        plot_interval=plot_interval, baseline=plot_baseline, out_sort=False, dpis=2000)

# Print all $\beta$ and Rs

In [11]:
taus = EXPERIMENTS['taus']
for r in r_experiments:
    for tau in taus:
        r_tau_res = exp_results[r][tau]
        any_key = list(r_tau_res.keys())[0]
        beta = exp_results[r][tau][any_key]['parameters']['beta']
        print('R: %s. Tau: %s. Beta: %s' % (r, tau, beta))

# Compare vaccination per age groups and regions

In [12]:
def plot_bars_ages_regions(ax, r, tau, metric, exp_results, labels, plot, u_offset,
                           t2=t2, t1=t1, baseline=plot_baseline, skip_no_vacc=True):
    # Get the results only for the specified r and tau
    results = exp_results[r][tau]
    
    # Count how many strategies we have
    total_strategies = len(results.keys())
    skip_num = 1
    if skip_no_vacc:
        skip_num += 1
    # Space for a set of bars (for one region or Age group)
    space_label = len_bar*(total_strategies-skip_num) + sep_bet_group
    
    # Find the strategy acting as baseline and get the results for it
    baseline_cumm = None
    for label, label_level in results.items():
        if label == baseline:
            baseline_results = label_level['results'][metric]
            baseline_cumm = baseline_results.cumsum(axis=2)

            if plot == 'Region':
                baseline_cumm = baseline_cumm.sum(axis=0)
            elif plot == 'Age group':
                baseline_cumm = baseline_cumm.sum(axis=1)
            else:
                raise ValueError('Not valid value to plot')
                
            baseline_cumm = baseline_cumm[:, u_offset:]
    
    assert baseline_cumm is not None
    
    # Add Finland to the bars
    labels = ['Finland'] + labels
    
    # Reserve space for the complete plot (all Regions or Age groups)
    total_plot_space = space_label*len(labels) + sep_bet_group
    ax.set_xlim(0, total_plot_space)
    
    # Iterate over the strategies, get result wrt baseline and plot
    num_strategy = 0
    legend_elements = []
    for label, label_level in results.items():
        if label == baseline:
            continue
        
        if label == 'No vaccination' and skip_no_vacc:
            continue

        # Get the results for this strategy
        policy_results = label_level['results'][metric]
        policy_cumm = policy_results.cumsum(axis=2)

        if plot == 'Region':
            policy_cumm = policy_cumm.sum(axis=0)
        elif plot == 'Age group':
            policy_cumm = policy_cumm.sum(axis=1)
        else:
            raise ValueError('Not valid value to plot')
        
        policy_cumm = policy_cumm[:, u_offset:]
                
        assert policy_cumm.shape == baseline_cumm.shape
        assert policy_cumm.shape[0] == len(labels) - 1

        # Obtain the the results for all the regions and age groups
        relative_to_baseline = policy_cumm/baseline_cumm
        relative_to_baseline = np.nan_to_num(relative_to_baseline, nan=1)
        relative_to_baseline = relative_to_baseline - 1
        relative_to_baseline = relative_to_baseline*100

        # Get the results between t1 and t2
        min_reduction = relative_to_baseline[:, t1]
        max_reduction = relative_to_baseline[:, t2]
        
        # Finland reduction
        finland_cumm = policy_cumm.sum(axis=0)
        base_finland_cumm = baseline_cumm.sum(axis=0)
        finland_to_baseline = finland_cumm/base_finland_cumm
        finland_to_baseline = np.nan_to_num(finland_to_baseline, nan=1)
        finland_to_baseline = finland_to_baseline - 1
        finland_to_baseline = finland_to_baseline*100
        
        min_red_finland = finland_to_baseline[t1]
        max_red_finland = finland_to_baseline[t2]
        
        min_reduction = np.insert(min_reduction, 0, min_red_finland)
        max_reduction = np.insert(max_reduction, 0, max_red_finland)
        
        assert min_reduction.shape == max_reduction.shape
        assert min_reduction.shape[0] == len(labels)

        color_i = num_strategy % len(color_palette)
        color = color_palette[color_i]
        # Iterating the regions or Age groups.
        for label_i in range(len(labels)):
            # Getting the the dimensions of the bar
            x1_square = label_i*space_label + sep_bet_group + num_strategy*len_bar
            y1_square = min_reduction[label_i]
            y2_square = max_reduction[label_i] - min_reduction[label_i]

            # Plot the result of this policy to the current region or Age group
            ax.broken_barh([(x1_square, len_bar), ], (y1_square, y2_square),
                           color=color)

        # Add one element to the legends for this strategy
        legend_elements.append(Line2D([0], [0], color=color, lw=legend_width, label=label))
        num_strategy += 1
    
    # Getting the positions for the ticks and labeling the labels for them
    space_bars = space_label - sep_bet_group
    middle_of_space = space_bars/2
    pos_tick =  sep_bet_group + middle_of_space
    ticks = []
    for label_i in range(len(labels)):
        abs_pos_tick = label_i*space_label + pos_tick
        ticks.append(abs_pos_tick)
        
    assert len(ticks) == len(labels)
    assert len(legend_elements) == total_strategies-skip_num
    
    # Add ticks and labels
    ax.set_xticks(ticks)
    ax.set_xticklabels(labels)
    ax.tick_params(labelsize=16)
    ax.set_xlabel(plot)
    ax.set_ylabel('Reduction in percentage')
    ax.set_title('Reduction in %s at %ss per policy . R=%s' % (metric, plot, r))
    ax.legend(handles=legend_elements, loc='upper left', framealpha=0.5)
    

    return ax

# Deaths

In [13]:
fig, ax = plt.subplots(figsize=(12, 10))
ax = plot_bars_ages_regions(ax, r=1.5, tau=0.5, metric='mortality', exp_results=exp_results,
                            labels=region_labels, u_offset=u_offset,
                            plot='Region', t2=t2, t1=t1, baseline=plot_baseline, skip_no_vacc=True)

In [14]:
fig, ax = plt.subplots(figsize=(12, 10))
ax = plot_bars_ages_regions(ax, r=1.5, tau=0.5, metric='mortality', exp_results=exp_results,
                            labels=age_labels, u_offset=u_offset,
                            plot='Age group', t2=t2, t1=t1, baseline=plot_baseline, skip_no_vacc=True)

# Hospitalizations

In [15]:
fig, ax = plt.subplots(figsize=(12, 10))
ax = plot_bars_ages_regions(ax, r=1.5, tau=0.5, metric='new hospitalizations', exp_results=exp_results,
                            labels=region_labels, u_offset=u_offset,
                            plot='Region', t2=t2, t1=t1, baseline=plot_baseline, skip_no_vacc=True)

In [16]:
fig, ax = plt.subplots(figsize=(12, 10))
ax = plot_bars_ages_regions(ax, r=1.5, tau=0.5, metric='new hospitalizations', exp_results=exp_results,
                            labels=age_labels, u_offset=u_offset,
                            plot='Age group', t2=t2, t1=t1, baseline=plot_baseline, skip_no_vacc=True)

# Infections

In [17]:
fig, ax = plt.subplots(figsize=(12, 10))
ax = plot_bars_ages_regions(ax, r=1.5, tau=0.5, metric='incidence', exp_results=exp_results,
                          labels=region_labels, u_offset=u_offset,
                          plot='Region', t2=t2, t1=t1, baseline=plot_baseline, skip_no_vacc=True)

In [18]:
fig, ax = plt.subplots(figsize=(12, 10))
ax = plot_bars_ages_regions(ax, r=1.5, tau=0.5, metric='incidence', exp_results=exp_results,
                            labels=age_labels, u_offset=u_offset,
                            plot='Age group', t2=t2, t1=t1, baseline=plot_baseline, skip_no_vacc=True)

# Saving figures and generating latex

In [19]:
def save_and_generate_bars(r_experiments, taus, region_labels, age_labels, metrics, exp_results,
                            dpis, u_offset,
                            skip_no_vacc, img_dir, t2, t1, baseline, file_format, latex_template):
    os.makedirs(img_dir, exist_ok=True)
    # Iterate over all results
    for r in r_experiments:
        for tau in taus:
            for metric in metrics:
                fig, axs = plt.subplots(2, 1, figsize=(11, 15))
                axs = axs.flat
                # Plot one figure for region and one for age group per r, tau pair
                for plot, labels, ax in zip(['Region', 'Age group'], [region_labels, age_labels], axs):
                    ax = plot_bars_ages_regions(ax=ax, r=r, tau=tau, metric=metric, plot=plot,
                                                u_offset=u_offset,
                                                exp_results=exp_results, labels=labels, t2=t2, t1=t1,
                                                baseline=baseline, skip_no_vacc=skip_no_vacc)
                    ax.set_ylabel('')
                    ax.set_title(None)
                # Setting the labels for the current figure
                y_lab = 'Change in percentage (\%)'
                fig.text(0.04, 0.5, y_lab, va='center', rotation='vertical', fontsize=16)
                # Saving the figure
                name = 'bars_r%s_tau%s_metric_%s' % (r, tau, metric)
                fig_name = name + '.' + file_format 
                fig_path = os.path.join(img_dir, fig_name)
                fig.savefig(fig_path, format=file_format, dpi=dpis, bbox_inches='tight')
                # Constructing Latex code for the current figure and printing it
                latext_text = latex_template.format(fig_name=fig_name,
                                                    metric=metric,
                                                    baseline=baseline,
                                                    name=name,
                                                    r=r,
                                                    tau=tau)
                print(latext_text)

In [20]:
latex_template = """
    \\begin{{figure}}[p]
      \\centering
      \\includegraphics[width=\\textwidth]{{img/{fig_name}}}
      \\caption{{Relative change in {metric} for all vaccination strategies with respect to vaccination strategy {baseline}.
      The bars represent the relative change in cumulative {metric} between the end of the simulation and 30 days before the end of the simulation.
      For this scenario, the effective reproduction number $\\Reff = {r}$ and the mobility value $\\tau = {tau}$.
      Top: Relative change at ERVA level, Bottom: Relative change at age group level.}}
      \\label{{fig:{name}}}
    \\end{{figure}}
"""
save_and_generate_bars(r_experiments=r_experiments, taus=[0.5], region_labels=region_labels,
                        age_labels=age_labels, u_offset=u_offset,
                        metrics=plot_metrics, exp_results=exp_results, skip_no_vacc=True,
                        img_dir=os.path.join('out', 'img'), t2=t2, t1=t1, baseline=plot_baseline,
                        file_format='pdf', latex_template=latex_template, dpis=2000)

# Generating table with all results

In [21]:
def generate_results_table(r_experiments, taus, metrics, baseline, exp_results, skip_no_vacc, col_order):
    table_dictionary = {}
    all_columns = set()
    # Iterate over all experiments
    for r in r_experiments:
        for tau in taus:
            # Group the results by r, tau
            key = (str(r), str(tau))
            table_dictionary[key] = {}
            results = exp_results[r][tau]
            # Iterate over all the metrics
            for metric in metrics:
                table_dictionary[key][metric] = {}
                # Iterate over all the strategies and get the counts
                for label, label_level in results.items():
                    if label == baseline:
                        baseline_results = label_level['results'][metric]
                        baseline_results = baseline_results.sum()
                        table_dictionary[key][metric]['baseline'] = {}
                        table_dictionary[key][metric]['baseline']['count'] = baseline_results
                    elif label == 'No vaccination' and skip_no_vacc:
                        continue
                    else:
                        # Add to columns since this is not the baseline
                        all_columns.add(label)
                        strategy_results = label_level['results'][metric]
                        strategy_results = strategy_results.sum()
                        table_dictionary[key][metric][label] = {}
                        table_dictionary[key][metric][label]['count'] = strategy_results
                        table_dictionary[key][metric][label]['best'] = False
    
    # Iterate over all r, tau combination
    for key, strategies_level in table_dictionary.items():
        # Iterate over all the metrics
        for metric in metrics:
            best_strategy_val = np.inf
            last_best = None
            # Iterate over all the strategies
            for column in all_columns:
                # Get the difference to the baseline
                strategies_level[metric][column]['diff_to_baseline'] = strategies_level[metric][column]['count'] - strategies_level[metric]['baseline']['count']
                # Get the relative change wrt to the baseline
                strategies_level[metric][column]['change_to_baseline'] = strategies_level[metric][column]['count']/strategies_level[metric]['baseline']['count'] - 1
                strategies_level[metric][column]['change_to_baseline'] = strategies_level[metric][column]['change_to_baseline']*100

                # Check if this strategy is the best for the r, tau combination
                if strategies_level[metric][column]['diff_to_baseline'] < best_strategy_val:
                    best_strategy_val = strategies_level[metric][column]['diff_to_baseline']
                    strategies_level[metric][column]['best'] = True
                    if last_best is not None:
                        strategies_level[metric][last_best]['best'] = False

                    last_best = column
                    
    assert set(col_order) == all_columns
    
    # Construct the latex tables
    header_latex = """
    \\begin{table}[h]
    \\centering
    \\scriptsize
    \\begin{tabular}{l%s}
    \\toprule
    """
    header_latex = header_latex % ('r'*len(all_columns))
    header_titles = ''
    for column in col_order:
        header_titles += '& \\bf %s ' % column
    complete_header = header_latex + header_titles + '\\\\ \n \\midrule \n'
    
    # Iterate over the results and add them to the latex table
    lines_diff = ''
    lines_change = ''
    for key, strategies_level in table_dictionary.items():
        r_str, tau_str = key
        line_diff = '$\\Reff=%s, \\tau=%s$ \\\\ \n' % (r_str, tau_str)
        line_change = line_diff
        for metric in metrics:
            line_diff += '\\quad %s ' % (metric.capitalize(), )
            line_change += '\\quad %s ' % (metric.capitalize(), )
            for column in col_order:
                if strategies_level[metric][column]['best']:
                    line_diff += '& \\bf '
                    line_change += '& \\bf '
                else:
                    line_diff += '& '
                    line_change += '& '
                line_diff += '%.2f ' % (round(strategies_level[metric][column]['diff_to_baseline'], 2))
                line_change += '%.2f ' % (round(strategies_level[metric][column]['change_to_baseline'], 2))
            line_diff += ' \\\\ \n'
            line_change += ' \\\\ \n'
        
        lines_diff += line_diff
        lines_change += line_change
    
    # Finish the tables with the captions
    tbl_diff = complete_header + lines_diff
    tbl_change = complete_header + lines_change
    
    end_table = """
    \\bottomrule
    \\end{tabular}
    """
    
    tbl_diff += end_table
    tbl_change += end_table
    
    tbl_diff += """
    \\caption{\\label{tab:ResultsRTauAbs} Absolute difference in mortality, hospital load, and incidence of
    different vaccination policies with respect to baseline (Pop) policy. Highest reductions are indicated in bold.}
    \\end{table}
    """
    
    tbl_change += """
    \\caption{\\label{tab:ResultsRTauRel} Relative difference in mortality, hospital load, and incidence of
    different vaccination policies with respect to baseline (Pop) policy. Highest reductions are indicated in bold.}
    \\end{table}
    """

    return tbl_diff, tbl_change

In [22]:
tbl_diff, tbl_change = generate_results_table(r_experiments=r_experiments, taus=taus, metrics=plot_metrics,
                                              baseline=plot_baseline, exp_results=exp_results,
                                              skip_no_vacc=True, col_order=col_order)
print(tbl_diff)
print(tbl_change)

# Custom plots

In [23]:
custom_metrics = ['mortality', 'new hospitalizations', 'incidence']
custom_rs = [1.5]
custom_tau = 0.5

fig, axs = plt.subplots(1, 3, sharex=True, figsize=(19, 8))

for i_r, r in enumerate(custom_rs):
    for i_m, metric in enumerate(custom_metrics):
            axs[i_m] = plot_results(axs[i_m], r=r, tau=custom_tau, metric=metric, T=T,
                              region='Finland', region_labels=[], exp_results=exp_results,
                              skip_no_vacc=True, u_offset=u_offset, plot_interval=plot_interval,
                              baseline=plot_baseline, out_sort=False)
            
            handles, labels = axs[i_m].get_legend_handles_labels()
            axs[i_m].get_legend().remove()
            x_lab = axs[i_m].get_xlabel()
            axs[i_m].set_xlabel('')
            y_lab = axs[i_m].get_ylabel()
            axs[i_m].set_ylabel('Relative change in {0} (\%)'.format(metric.capitalize()), fontsize=16)
            axs[i_m].set_title(None)

fig.text(0.5, -0.02, x_lab, ha='center', fontsize=24)
ncols = round(len(labels)/2)
legend = fig.legend(handles, labels, fontsize=18,
                    loc='upper center', bbox_to_anchor=(0.5, 1.1),
                    framealpha=0.5, ncol=ncols)
fig.tight_layout()
fig_name = 'custom_finland.pdf'
img_dir = os.path.join('out', 'img')
fig_path = os.path.join(img_dir, fig_name)
fig.savefig(fig_path, format='pdf', dpi=2000, bbox_inches='tight')

In [24]:
custom_rs = [1.25, 1.5]

fig, axs = plt.subplots(2, 1, figsize=(11, 15))

for i_r, r in enumerate(custom_rs):
    axs[i_r] = plot_bars_ages_regions(ax=axs[i_r], r=r, tau=0.5, metric='mortality', plot='Region',
                              u_offset=u_offset, exp_results=exp_results, labels=region_labels, t2=t2, t1=t1,
                              baseline=plot_baseline, skip_no_vacc=True)
    axs[i_r].set_ylabel('')
    axs[i_r].set_title(None)
# Setting the labels for the current figure
y_lab = 'Change in percentage (\%)'
fig.text(0.04, 0.5, y_lab, va='center', rotation='vertical', fontsize=16)
fig_name = 'bars_custom.pdf'
img_dir = os.path.join('out', 'img')
fig_path = os.path.join(img_dir, fig_name)
fig.savefig(fig_path, format='pdf', dpi=2000, bbox_inches='tight')